In [1]:
import json
import os

In [8]:
# Load two files:
model_outputs = json.load(open('/Users/venkatakesavvenna/Sem6/RSAI/Mid_Project/dataset/nli_output/mistral_7b_checker_final.json'))
human_outputs = json.load(open('/Users/venkatakesavvenna/Sem6/RSAI/Mid_Project/dataset/human_annotations/zero_context/nq_mistral_7b_answers.json'))

In [9]:
# Print the keys 
print(model_outputs[0].keys())
print(human_outputs[0].keys())

dict_keys(['id', 'response', 'triplets', 'reference', 'Y', 'ys'])
dict_keys(['id', 'response', 'claude2_response_kg'])


In [10]:
# Make an array of the form [triplet, model_output, human_output]
final_array = []

# Check the ids, that are common in both the files
for model_output in model_outputs:
    for human_output in human_outputs:
        if model_output['id'] == human_output['id']:
            final_array.append([model_output["triplets"], model_output["ys"], [entry["human_label"] for entry in human_output["claude2_response_kg"]]])
            

In [11]:
from sklearn.metrics import confusion_matrix
import numpy as np

# Categories
categories = ['Neutral', 'Entailment', 'Contradiction']

# Initialize confusion matrix
conf_matrix = np.zeros((len(categories), len(categories)))

# Count occurrences of each combination of model and human outputs
for entry in final_array:
    model_outputs = entry[1]
    human_outputs = entry[2]

    for model_output, human_output in zip(model_outputs, human_outputs):
        # Get indices of categories
        model_index = categories.index(model_output)
        human_index = categories.index(human_output)

        # Update confusion matrix
        conf_matrix[model_index][human_index] += 1

# Print confusion matrix
print("Confusion Matrix:")
print(conf_matrix)

# Calculate accuracy
accuracy = np.trace(conf_matrix) / np.sum(conf_matrix)
print(f"\nAccuracy: {accuracy}")

# Calculate precision, recall, and F1-score for each category
for i, category in enumerate(categories):
    true_positives = conf_matrix[i][i]
    false_positives = sum(conf_matrix[:, i]) - true_positives
    false_negatives = sum(conf_matrix[i, :]) - true_positives

    precision = true_positives / (true_positives + false_positives)
    recall = true_positives / (true_positives + false_negatives)
    f1_score = 2 * (precision * recall) / (precision + recall)

    print(f"\nCategory: {category}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1-score: {f1_score}")


Confusion Matrix:
[[22.  3.  9.]
 [ 8. 59.  9.]
 [ 6.  4. 43.]]

Accuracy: 0.7607361963190185

Category: Neutral
Precision: 0.6111111111111112
Recall: 0.6470588235294118
F1-score: 0.6285714285714287

Category: Entailment
Precision: 0.8939393939393939
Recall: 0.7763157894736842
F1-score: 0.8309859154929577

Category: Contradiction
Precision: 0.7049180327868853
Recall: 0.8113207547169812
F1-score: 0.7543859649122806


In [12]:
# Analysis for Alpaca-7B model
model_outputs = json.load(open('dataset/human_annotations/zero_context/nq_alpaca_7B_answers.json'))
human_outputs = json.load(open('dataset/nli_output/alpaca_7b_checker_final.json'))

# Make an array of the form [triplet, model_output, human_output]
final_array = []

# Take the majority votes for the human outputs
for model_output in model_outputs:
    for human_output in human_outputs:
        if model_output['id'] == human_output['id']:
            model_final = [max(set(model_output["ys"]), key = model_output["ys"].count)]
            human_final = [max(set([entry["human_label"] for entry in human_output["claude2_response_kg"]]), key = [entry["human_label"] for entry in human_output["claude2_response_kg"]].count)]

            final_array.append([model_output["triplets"], model_final, human_final])

# Check if the models, human outputs are matching
matching = 0

for entry in final_array:
    if entry[1] == entry[2]:
        matching += 1

print(f"Matching: {matching} out of {len(final_array)}")

FileNotFoundError: [Errno 2] No such file or directory: '/Users/venkatakesavvenna/Sem6/RSAI/Mid_Project/dataset/nli_output/alpaca-7b_checker_final.json'